In [ ]:
import tensorflow as tf

def lrelu(X, leak=0.2):
    f1 = 0.5 * (1 + leak)
    f2 = 0.5 * (1 - leak)
    return f1 * X + f2 * tf.abs(X)

def generator(x, ngf=64, isTrain=True, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
        b_init = tf.constant_initializer(0.0)

        conv1 = tf.layers.conv2d(x, ngf, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init)
        conv2 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv1, 0.2), ngf * 2, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv3 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv2, 0.2), ngf * 4, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv4 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv3, 0.2), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv5 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv4, 0.2), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv6 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv5, 0.2), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv7 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv6, 0.2), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv8 = tf.layers.conv2d(lrelu(conv7, 0.2), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init)

        deconv1 = tf.nn.dropout(tf.layers.batch_normalization(tf.layers.conv2d_transpose(tf.nn.relu(conv8), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain), keep_prob=0.5)
        deconv1 = tf.concat([deconv1, conv7], 3)
        deconv2 = tf.nn.dropout(tf.layers.batch_normalization(tf.layers.conv2d_transpose(tf.nn.relu(deconv1), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain), keep_prob=0.5)
        deconv2 = tf.concat([deconv2, conv6], 3)
        deconv3 = tf.nn.dropout(tf.layers.batch_normalization(tf.layers.conv2d_transpose(tf.nn.relu(deconv2), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain), keep_prob=0.5)
        deconv3 = tf.concat([deconv3, conv5], 3)
        deconv4 = tf.layers.batch_normalization(tf.layers.conv2d_transpose(tf.nn.relu(deconv3), ngf * 8, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        deconv4 = tf.concat([deconv4, conv4], 3)
        deconv5 = tf.layers.batch_normalization(tf.layers.conv2d_transpose(tf.nn.relu(deconv4), ngf * 4, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        deconv5 = tf.concat([deconv5, conv3], 3)
        deconv6 = tf.layers.batch_normalization(tf.layers.conv2d_transpose(tf.nn.relu(deconv5), ngf * 2, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        deconv6 = tf.concat([deconv6, conv2], 3)
        deconv7 = tf.layers.batch_normalization(tf.layers.conv2d_transpose(tf.nn.relu(deconv6), ngf, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        deconv7 = tf.concat([deconv7, conv1], 3)
        deconv8 = tf.layers.conv2d_transpose(tf.nn.relu(deconv7), 3, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init)

        o = tf.nn.tanh(deconv8)

        return o

def discriminator(x, y, ndf=64, isTrain=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):

        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
        b_init = tf.constant_initializer(0.0)

        cat1 = tf.concat([x, y], 3) 
        conv1 = tf.layers.conv2d(cat1, ndf, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init)
        conv2 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv1, 0.2), ndf * 2, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv3 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv2, 0.2), ndf * 4, [4, 4], strides=(2, 2), padding='same', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv3 = tf.pad(conv3, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
        conv4 = tf.layers.batch_normalization(tf.layers.conv2d(lrelu(conv3, 0.2), ndf * 8, [4, 4], strides=(1, 1), padding='valid', kernel_initializer=w_init, bias_initializer=b_init), training=isTrain)
        conv4 = tf.pad(conv4, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
        conv5 = tf.layers.conv2d(lrelu(conv4, 0.2), 1, [4, 4], strides=(1, 1), padding='valid', kernel_initializer=w_init, bias_initializer=b_init)

        o = tf.nn.sigmoid(conv5)

        return o, conv5

In [ ]:
import itertools, imageio, os, random
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imresize

def show_result(x_, imgs, y_, num_epoch, show = False, save = False, path = 'result.png'):
    size_figure_grid = 3
    fig, ax = plt.subplots(x_.shape[0], size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(x_.shape[0]), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for i in range(x_.shape[0]):
        ax[i, 0].cla()
        ax[i, 0].imshow(denorm(x_[i]) / 255.0)
        ax[i, 1].cla()
        ax[i, 1].imshow(denorm(imgs[i]) / 255.0)
        ax[i, 2].cla()
        ax[i, 2].imshow(denorm(y_[i]) / 255.0)

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Iter')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

def generate_animation(root, model, opt):
    images = []
    for e in range(opt.train_epoch):
        img_name = root + 'Fixed_results/' + model + str(e + 1) + '.png'
        images.append(imageio.imread(img_name))
    imageio.mimsave(root + model + 'generate_animation.gif', images, fps=5)

def imgs_resize(imgs, resize_scale = 286):
    outputs = np.zeros((imgs.shape[0], resize_scale, resize_scale, imgs.shape[3]))
    for i in range(imgs.shape[0]):
        img = imresize(imgs[i], [resize_scale, resize_scale])
        outputs[i] = img.astype(np.float32)

    return outputs

def random_crop(imgs1, imgs2, crop_size = 256):
    outputs1 = np.zeros((imgs1.shape[0], crop_size, crop_size, imgs1.shape[3]))
    outputs2 = np.zeros((imgs2.shape[0], crop_size, crop_size, imgs2.shape[3]))
    for i in range(imgs1.shape[0]):
        img1 = imgs1[i]
        img2 = imgs2[i]
        rand1 = np.random.randint(0, imgs1.shape[1] - crop_size)
        rand2 = np.random.randint(0, imgs2.shape[1] - crop_size)
        outputs1[i] = img1[rand1: crop_size + rand1, rand2: crop_size + rand2, :]
        outputs2[i] = img2[rand1: crop_size + rand1, rand2: crop_size + rand2, :]

    return outputs1, outputs2

def random_fliplr(imgs1, imgs2):
    outputs1 = np.zeros(imgs1.shape)
    outputs2 = np.zeros(imgs2.shape)
    for i in range(imgs1.shape[0]):
        if np.random.rand(1) < 0.5:
            outputs1[i] = np.fliplr(imgs1[i])
            outputs2[i] = np.fliplr(imgs2[i])
        else:
            outputs1[i] = imgs1[i]
            outputs2[i] = imgs2[i]

    return outputs1, outputs2

def norm(img):
    return (img - 127.5) / 127.5

def denorm(img):
    return (img * 127.5) + 127.5

class data_loader:
    def __init__(self, root, batch_size=1, shuffle=False):
        self.root = root
        self.batch_size = batch_size
        self.file_list = os.listdir(self.root)
        if shuffle:
            self.file_list = list(np.array(self.file_list)[random.sample(range(0, len(self.file_list)), len(self.file_list))])
        img = plt.imread(self.root + '/' + self.file_list[0])
        self.shape = (len(self.file_list), img.shape[0], img.shape[1], img.shape[2])
        self.flag = 0

    def next_batch(self):
        if self.flag + self.batch_size > self.shape[0]:
            self.file_list = list(np.array(self.file_list)[random.sample(range(0, len(self.file_list)), len(self.file_list))])
            self.flag = 0

        output = np.zeros((self.batch_size, self.shape[1], self.shape[2], self.shape[3]))
        temp = 0
        for i in range(self.flag, self.flag + self.batch_size):
            output[temp] = plt.imread(self.root + '/' + self.file_list[i])
            temp = temp + 1

        self.flag += self.batch_size

        return output

In [ ]:
import os, time, pickle, easydict
import tensorflow as tf
import numpy as np

opt= easydict.EasyDict({ "dataset": 'steels', 
                        "train_subfolder": 'train',
                        "test_subfolder": 'val',
                        "batch_size":1,
                        "test_batch_size": 2,
                        "ngf":64,
                        "ndf":64,
                        "input_size":256,
                        "crop_size":256,
                        "resize_scale":286,
                        "fliplr":True,
                        "train_epoch":200,
                        "lrD":0.0002,
                        "lrG":0.0002,
                        "L1_lambda":100,
                        "beta1":0.5,
                        "beta2":0.999,
                        "save_root":'results',
                        "inverse_order":True
                       })

root = opt.dataset + '_' + opt.save_root + '/'
model = opt.dataset + '_'
if not os.path.isdir(root):
    os.mkdir(root)
if not os.path.isdir(root + 'Fixed_results'):
    os.mkdir(root + 'Fixed_results')

train_loader = data_loader('D:/SEM and OM pix2pix data/data/' + opt.dataset + '/' + opt.train_subfolder, opt.batch_size, shuffle=True)
test_loader = data_loader('D:/SEM and OM pix2pix data/data/' + opt.dataset + '/' + opt.test_subfolder, opt.test_batch_size, shuffle=True)
img_size = train_loader.shape[1]
test_img = test_loader.next_batch()
if opt.inverse_order:
    fixed_x_ = test_img[:, :, img_size:, :]
    fixed_y_ = test_img[:, :, 0:img_size, :]
else:
    fixed_x_ = test_img[:, :, 0:img_size, :]
    fixed_y_ = test_img[:, :, img_size:, :]

if img_size != opt.input_size:
    fixed_x_ = imgs_resize(fixed_x_, opt.input_size)
    fixed_y_ = imgs_resize(fixed_y_, opt.input_size)

fixed_x_ = norm(fixed_x_)
fixed_y_ = norm(fixed_y_)

x = tf.placeholder(tf.float32, shape=(None, opt.input_size, opt.input_size, train_loader.shape[3]))
y = tf.placeholder(tf.float32, shape=(None, opt.input_size, opt.input_size, train_loader.shape[3]))

G = generator(x, opt.ngf)
D_positive, D_positive_logits = discriminator(x, y, opt.ndf)
D_negative, D_negative_logits = discriminator(x, G, opt.ndf, reuse=True)

D_loss_positive = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_positive_logits, labels=tf.ones_like(D_positive_logits)))
D_loss_negative = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_negative_logits, labels=tf.zeros_like(D_negative_logits)))
D_loss = (D_loss_positive + D_loss_negative) * 0.5
G_loss_gan = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_negative_logits, labels=tf.ones_like(D_negative_logits)))
G_loss_L1 = tf.reduce_mean(tf.reduce_sum(tf.abs(G - y), 3))
G_loss = G_loss_gan + opt.L1_lambda * G_loss_L1

T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator')]

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_optim = tf.train.AdamOptimizer(opt.lrD, beta1=opt.beta1, beta2=opt.beta2).minimize(D_loss, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(opt.lrG, beta1=opt.beta1, beta2=opt.beta2).minimize(G_loss, var_list=G_vars)

saver = tf.train.Saver()
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

print('training start!')
start_time = time.time()
for epoch in range(opt.train_epoch):
    D_losses = []
    G_losses = []
    epoch_start_time = time.time()
    num_iter = 0
    for iter in range(train_loader.shape[0] // opt.batch_size):
        train_img = train_loader.next_batch()

        if opt.inverse_order:
            x_ = train_img[:, :, img_size:, :]
            y_ = train_img[:, :, 0:img_size, :]
        else:
            x_ = train_img[:, :, 0:img_size, :]
            y_ = train_img[:, :, img_size:, :]

        if img_size != opt.input_size:
            x_ = imgs_resize(x_, opt.input_size)
            y_ = imgs_resize(y_, opt.input_size)

        if opt.resize_scale:
            x_ = imgs_resize(x_, opt.resize_scale)
            y_ = imgs_resize(y_, opt.resize_scale)

        if opt.crop_size:
            x_, y_ = random_crop(x_, y_, opt.crop_size)

        if opt.fliplr:
            x_, y_ = random_fliplr(x_, y_)

        x_ = norm(x_)
        y_ = norm(y_)

        loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, y: y_})
        D_losses.append(loss_d_)
        train_hist['D_losses'].append(loss_d_)

        loss_g_, _ = sess.run([G_loss, G_optim], {x: x_, y: y_})
        G_losses.append(loss_g_)
        train_hist['G_losses'].append(loss_g_)

        num_iter += 1

    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time

    print('[%d/%d] - ptime: %.2f, loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), opt.train_epoch, per_epoch_ptime, np.mean((D_losses)), np.mean(G_losses)))
    fixed_p = root + 'Fixed_results/' + model + str(epoch + 1) + '.png'
    outputs = sess.run(G, {x: fixed_x_})
    show_result(fixed_x_, outputs, fixed_y_, (epoch+1), save=True, path=fixed_p)
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)

print("Avg. one epoch ptime: %.2f, total %d epochs ptime: %.2f" % (np.mean(train_hist['per_epoch_ptimes']), opt.train_epoch, total_ptime))
print("Training finish!... save training results")
with open(root + model + 'train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

saver.save(sess, root + model + 'params.ckpt')

show_train_hist(train_hist, save=True, path=root + model + 'train_hist.png')

#pix2pix_training_code

In [ ]:
import os, time, easydict
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

opt= easydict.EasyDict({ "dataset": 'steels',
                        "input_size":256,
                        "test_subfolder": 'val',
                        "ngf":64,
                        "ndf":64,
                        "save_root":'results',
                        "inverse_order":True
                       })

if not os.path.isdir(opt.dataset + '_results/test_results'):
    os.mkdir(opt.dataset + '_results/test_results')

test_loader = data_loader('D:/SEM and OM pix2pix data/data/' + opt.dataset + '/' + opt.test_subfolder, 1, shuffle=False)
img_size = test_loader.shape[1]

x = tf.placeholder(tf.float32, shape=(None, opt.input_size, opt.input_size, test_loader.shape[3]))

G = generator(x, opt.ngf)

saver = tf.train.Saver()
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
saver.restore(sess, (opt.dataset + '_results/steels_params.ckpt'))

print('test start!')

per_ptime = []
total_start_time = time.time()
for iter in range(test_loader.shape[0]):
    per_start_time = time.time()

    train_img = test_loader.next_batch()

    if opt.inverse_order:
        x_ = train_img[:, :, img_size:, :]
        y_ = train_img[:, :, 0:img_size, :]
    else:
        x_ = train_img[:, :, 0:img_size, :]
        y_ = train_img[:, :, img_size:, :]

    if img_size != opt.input_size:
        x_ = imgs_resize(x_, opt.input_size)
        y_ = imgs_resize(y_, opt.input_size)

    x_ = norm(x_)
    y_ = norm(y_)

    test_img = sess.run(G, {x: x_})

    num_str = test_loader.file_list[iter][:test_loader.file_list[iter].find('.')]
    path = opt.dataset + '_results/test_results/' + num_str + '_input.png'
    plt.imsave(path, (denorm(x_[0]) / 255))
    path = opt.dataset + '_results/test_results/' + num_str + '_output.png'
    plt.imsave(path, (denorm(test_img[0]) / 255))
    path = opt.dataset + '_results/test_results/' + num_str + '_target.png'

    per_end_time = time.time()
    per_ptime.append(per_end_time - per_start_time)

total_end_time = time.time()
total_ptime = total_end_time - total_start_time

print('total %d images generation complete!' % (iter+1))
print('Avg. one image process ptime: %.2f, total %d images process ptime: %.2f' % (np.mean(per_ptime), (iter+1), total_ptime))

#pix2pix_test_code